In [1]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [2]:
import json
import re
import requests
from pathlib import Path

import pandas as pd
from tqdm.auto import tqdm
import os

In [7]:
project_dir = Path(os.path.abspath('')).parents[1]
data_dir = project_dir / "data"

In [23]:
pubs = pd.read_csv(data_dir / "interim/queried_ids.csv", index_col="article_id")

p = pubs[pubs.CR_DOI_score.notna()]
p.head()

,proposal_id,unstruct_citation,type,DOI,PMCID,PMID,URL,year,container-title,title,author_str,CR_DOI_candidate,CR_DOI_score
article_id,,,,,,,,,,,,,
160,13183003,"Mondello S, Shear DA, Bramlett HM, et al. 2016...",PH_TBI,10.1089/neu.2015.4132.PMID,NaN,NaN,NaN,2016.0,J Neurotrauma,Insight into Pre-Clinical Models of Traumatic ...,"Mondello, S.; Shear, D.A.; Bramlett, H.M.",10.1089/neu.2015.4132,106.854430
293,PT073555,"Zhang L, Li H, Hu X, et al. 2011. Glucocortico...",PH_TBI,NaN,NaN,NaN,NaN,2011.0,Med Hypotheses,Glucocorticoid-induced p11 over-expression and...,"Zhang, L.; Li, H.; Hu, X.",10.1016/j.mehy.2011.02.015,117.713720
464,PT074693,"Shao Y, Keliris GA, Papanikolaou A, et al. 201...",PH_TBI,NaN,NaN,NaN,NaN,2013.0,Eur J Neurosci,Visual cortex organisation in a macaque monkey...,"Shao, Y.; Keliris, G.A.; Papanikolaou, A.",10.1111/ejn.12349,94.128510
622,PT074968,"Arenth PM, Russell KC, Ricker JH, et al. 2011....",PH_TBI,NaN,NaN,NaN,NaN,2011.0,Phys Med Rehabil,CDP-choline as a biological supplement during ...,"Arenth, P.M.; Russell, K.C.; Ricker, J.H.",10.1016/j.pmrj.2011.03.012,108.252205
743,PT075791,"Hartings JA, Bullock MR, Okonkwo DO, et al. 20...",PH_TBI,NaN,NaN,NaN,NaN,2011.0,Lancet Neurology,Spreading depolarizations and outcome after tr...,"Hartings, J.A.; Bullock, M.R.; Okonkwo, D.O.",10.1016/s1474-4422(11)70243-5,78.840170


In [24]:
min_score = 80
p[p.CR_DOI_score >= 80]

,proposal_id,unstruct_citation,type,DOI,PMCID,PMID,URL,year,container-title,title,author_str,CR_DOI_candidate,CR_DOI_score
article_id,,,,,,,,,,,,,
160,13183003,"Mondello S, Shear DA, Bramlett HM, et al. 2016...",PH_TBI,10.1089/neu.2015.4132.PMID,NaN,NaN,NaN,2016.0,J Neurotrauma,Insight into Pre-Clinical Models of Traumatic ...,"Mondello, S.; Shear, D.A.; Bramlett, H.M.",10.1089/neu.2015.4132,106.854430
293,PT073555,"Zhang L, Li H, Hu X, et al. 2011. Glucocortico...",PH_TBI,NaN,NaN,NaN,NaN,2011.0,Med Hypotheses,Glucocorticoid-induced p11 over-expression and...,"Zhang, L.; Li, H.; Hu, X.",10.1016/j.mehy.2011.02.015,117.713720
464,PT074693,"Shao Y, Keliris GA, Papanikolaou A, et al. 201...",PH_TBI,NaN,NaN,NaN,NaN,2013.0,Eur J Neurosci,Visual cortex organisation in a macaque monkey...,"Shao, Y.; Keliris, G.A.; Papanikolaou, A.",10.1111/ejn.12349,94.128510
622,PT074968,"Arenth PM, Russell KC, Ricker JH, et al. 2011....",PH_TBI,NaN,NaN,NaN,NaN,2011.0,Phys Med Rehabil,CDP-choline as a biological supplement during ...,"Arenth, P.M.; Russell, K.C.; Ricker, J.H.",10.1016/j.pmrj.2011.03.012,108.252205
781,PT075827,"Bista S, Zhuo J, Gullapalli RP, et al. 2014. V...",PH_TBI,NaN,NaN,NaN,NaN,2014.0,IEEE Transactions on Visualization and Compute...,Visualization of brain microstructure through ...,"Bista, S.; Zhuo, J.; Gullapalli, R.P.",10.1109/tvcg.2014.2346411,132.209210
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18080,RP950603,Gewirtz DA (2000): Growth arrest and cell deat...,BC,NaN,NaN,NaN,NaN,2000.0,Breast Cancer Research and Treatment,Growth arrest and cell death in the breast tum...,"Gewirtz, D.A.",10.1023/a:1006414422919,114.451300
18131,RP950688,"Zhang H, Serrero G (1998): Inhibition of tumor...",BC,NaN,NaN,NaN,NaN,1998.0,Proceedings of the National Academy of Science...,Inhibition of tumorigenicity of the teratoma P...,"Zhang, H.; Serrero, G.",10.1073/pnas.95.24.14202,162.337700
18319,RP951156,Felten DL (2000): Neural influence on immune r...,BC,NaN,NaN,NaN,NaN,2000.0,Progress in Brain Research,Neural influence on immune responses: underlyi...,"Felten, D.L.",10.1016/s0079-6123(08)62152-4,110.121290
